In [1]:
import dask
import dask.dataframe as dd
from dask import delayed

In [2]:
import sys
sys.path.append('/Users/pradap/Documents/Research/Python-Package/scaling/dask_em')

In [3]:
import string

import pandas as pd
from dask import threaded, delayed
from dask_em.sampler.downsample.dsprober import DownSampleProber

from dask_em.utils.cy_utils.stringcontainer import StringContainer
from dask_em.utils.py_utils.utils import get_str_cols, str2bytes, sample, split_df, \
    tokenize_strings_wsp, build_inv_index


In [4]:
def preprocess_table(dataframe, idcol):
    strcols = list(get_str_cols(dataframe))
    strcols.append(idcol)
    projdf = dataframe[strcols]
    objsc = StringContainer()
    for row in projdf.itertuples():
        colvalues = row[1:-1]
        uid = row[-1]
        strings = [colvalue.strip() for colvalue in colvalues if not pd.isnull(colvalue)]
        concat_row = str2bytes(' '.join(strings).lower())
        concat_row = concat_row.translate(None, string.punctuation)
        objsc.push_back(uid, concat_row)
    return objsc

def probe(objtc, objinvindex, yparam):
    objprobe = DownSampleProber()
    objprobe.probe(objtc, objinvindex, yparam)
    return objprobe

# @delayed(nout=2)
def lpostprocess(result_list):
    lids = set()
#     rids = set()
    for i in range(len(result_list)):
        result = result_list[i]
        lids.update(result.get_lids())
#         rids.update(result.get_rids())
    lids = sorted(lids)
#     rids = sorted(rids)
    return lids

def rpostprocess(result_list):
#     lids = set()
    rids = set()
    for i in range(len(result_list)):
        result = result_list[i]
#         lids.update(result.get_lids())
        rids.update(result.get_rids())
#     lids = sorted(lids)
    rids = sorted(rids)
    return rids


def select(table, ids):
    return table.loc[ids]


In [5]:
path = '../datasets/'

In [6]:
A = dd.read_csv(path+'msd.csv')
B = dd.read_csv(path+'msd.csv')
A.set_index('id', drop=False)
B.set_index('id', drop=False)

,id,title,release,artist_name,duration,artist_familiarity,artist_hotttnesss,year
npartitions=2,,,,,,,,
1,int64,object,object,object,float64,float64,float64,int64
608399,...,...,...,...,...,...,...,...
1000000,...,...,...,...,...,...,...,...


In [7]:
vals = [(0, '', '', '', 0.0, 0.0, 0.0, 1)]
cols=[u'id', u'title', u'release', u'artist_name', u'duration',u'artist_familiarity', u'artist_hotttnesss', u'year']
meta = pd.DataFrame(vals, columns=cols)

In [14]:
A['id'].dtype

dtype('int64')

In [15]:
A.dtypes

id                      int64
title                  object
release                object
artist_name            object
duration              float64
artist_familiarity    float64
artist_hotttnesss     float64
year                    int64
dtype: object

In [19]:
vals = []
cols = []
for col in A.columns:
    if A[col].dtype == 'int64':
        vals.append(0)
    elif A[col].dtype == 'float64':
        vals.append(0.0)
    elif A[col].dtype == 'bool':
        vals.append(False)
    elif A[col].dtype == 'object':
        vals.append('')
    else:
        raise ValueError(col)
result = [tuple(vals)]
meta = pd.DataFrame(result, columns=list(A.columns))

        
    
    

In [20]:
meta

,id,title,release,artist_name,duration,artist_familiarity,artist_hotttnesss,year
0,0,,,,0.0,0.0,0.0,0


In [ ]:
def get_meta(table):
    for 

In [19]:
lid = 'id'
rid = 'id'
lstopwords = ['the', 'of', 'in', 'and', 'version', 'and', 'to', 'you', 'i', 'love', 'me', 'album', 'live', 'my', 'on', 'for', 'de', 'it', 'best', 'is', 'la', 'vol', 'from', 'no', 'your', 'all']

rstopwords = lstopwords
stopwords = ['a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its', 'on', 'that', 'the', 'to', 'of', 'was', 'were', 'will', 'with']

# stopwords


lstopwords = lstopwords + stopwords
rstopwords = rstopwords + stopwords

ltokens = []
for i in range(A.npartitions):
    lcat_strings = (delayed)(preprocess_table)(A.get_partition(i), lid)
    tokens = (delayed)(tokenize_strings_wsp)(lcat_strings, lstopwords)
    ltokens.append(tokens)

    
    
invindex = (delayed)(build_inv_index)(ltokens)
rsample = B.sample(0.1, random_state=0)

probe_rslts = []
y = 1
for i in range(rsample.npartitions):
    rcat_strings = (delayed)(preprocess_table)(rsample.get_partition(i), rid)
    rtokens = (delayed)(tokenize_strings_wsp)(rcat_strings, rstopwords)
    probe_rslt = (delayed)(probe)(rtokens, invindex, y)
    probe_rslts.append(probe_rslt)

lresult = (delayed)(lpostprocess)(probe_rslts)
rresult = (delayed)(rpostprocess)(probe_rslts)

resA = []
for i in range(A.npartitions):
    tmp = (delayed)(select)(A.get_partition(i), lresult)
    resA.append(tmp)
A1 = dd.from_delayed(resA, meta=meta)

resB = []
for i in range(rsample.npartitions):
    tmp = (delayed)(select)(rsample.get_partition(i), rresult)
    resB.append(tmp)
B1 = dd.from_delayed(resB, meta=meta)


    
# A1 = delayed(select)(A, lresult, meta)
# B1 = delayed(select)(rsample, rresult, meta)
# B1 = rsample.map_partitions(lambda x: x[x.index.isin(rresult)], meta = meta) 


In [77]:
# !mkdir s1
# !mkdir s2

In [16]:
from dask.diagnostics import *

In [21]:
with Profiler() as prof, ResourceProfiler(dt=0.25) as rprof, CacheProfiler() as cprof: 
    len(rsample), len(A1), len(B1)

In [22]:
visualize([prof, rprof, cprof])

Column(id='4d0bfd97-26c7-4709-8e0b-9a6157433b7e', ...)

In [63]:
A1.head()

Delayed('head-3cb94cc6-40a3-4176-9e55-7cdf39e3b795')

SyntaxError: invalid syntax (<ipython-input-31-d1fc8db7b247>, line 1)

In [55]:
l = [1, 2, 3, 4, 5, 7, 8, 10, 12, 35, 46]

In [50]:
C = A.set_index('id', drop=False)

In [59]:
C.columns

Index([u'id', u'title', u'release', u'artist_name', u'duration',
       u'artist_familiarity', u'artist_hotttnesss', u'year'],
      dtype='object')

In [ ]:
import pandas as pd

In [68]:
pd.DataFrame(

,h
0,1
1,2
2,3


In [75]:
vals = [(0, '', '', '', 0.0, 0.0, 0.0, 1)]
cols=[u'id', u'title', u'release', u'artist_name', u'duration',u'artist_familiarity', u'artist_hotttnesss', u'year']

In [77]:
meta=pd.DataFrame(vals, columns=cols)

In [78]:
X = C.map_partitions(lambda x: x[x.index.isin(l)], meta = meta)

In [80]:
X.head()

,id,title,release,artist_name,duration,artist_familiarity,artist_hotttnesss,year
id,,,,,,,,
1,1,Silent Night,Monster Ballads X-Mas,Faster Pussy cat,252.05506,0.649822,0.394032,2003
2,2,Tanssi vaan,Karkuteillä,Karkkiautomaatti,156.55138,0.439604,0.356992,1995
3,3,No One Could Ever,Butter,Hudson Mohawke,138.97098,0.643681,0.437504,2006
4,4,Si Vos Querés,De Culo,Yerba Brava,145.05751,0.448501,0.372349,2003
5,5,Tangle Of Aspens,Rene Ablaze Presents Winter Sessions,Der Mystic,514.29832,0.000000,0.000000,0
